
# 🩺 PhysioNet – ECG **Image Digitization + Reconstruction** (Hybrid)

> 💡 **If this notebook helps you understand ECG reconstruction and image digitization, please consider giving it an upvote.**
> It helps others discover it and keeps me motivated to share more.

**What this does:**  
- Extracts **12-lead time-series** directly from **paper ECG images** (scans/photos) using a fast OpenCV pipeline  
- Builds **per-lead median templates** from training CSVs  
- Reconstructs test signals via **BPM-tiling + micro-ensemble**  
- **Hybrid blend:** combines the image-digitized traces with the template-based reconstruction  
- Enforces **Einthoven’s law** for limb-lead consistency  
- Writes a valid **`submission.csv`**

📦 **Kaggle constraints:** No internet, ≤9h runtime.  



## 📑 Contents
1. [Config](#config)  
2. [Utilities (Signals)](#utils-signals)  
3. [Utilities (Images)](#utils-images)  
4. [Template Building from Training CSVs](#templates)  
5. [Reconstruction (BPM tiling + Ensemble + Einthoven)](#recon)  
6. [Image Digitization (OpenCV pipeline + model hook)](#digitize)  
7. [Hybrid Blending](#hybrid)  
8. [Predict & Create `submission.csv`](#submission)  
9. [Visual Checks](#viz)  
10. [Notes & Next Steps](#next)


## ⚙️ Step 1 — Config

In [ ]:

import os, math, random, glob, cv2
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from scipy.signal import butter, filtfilt, find_peaks

# Kaggle paths
INPUT_DIR = Path("/kaggle/input/physionet-ecg-image-digitization")
TRAIN_CSV = INPUT_DIR / "train.csv"
TRAIN_DIR = INPUT_DIR / "train"
TEST_CSV  = INPUT_DIR / "test.csv"
TEST_IMG_DIR = INPUT_DIR / "test_images"

# Output
WORK_DIR = Path("/kaggle/working")
WORK_DIR.mkdir(parents=True, exist_ok=True)

# Leads & defaults
LEADS = ["I","II","III","aVR","aVL","aVF","V1","V2","V3","V4","V5","V6"]

# Reconstruction params
R_PRE_S, R_POST_S = 0.20, 0.40
BEAT_LEN = 360
BP_LO_HZ, BP_HI_HZ, BP_ORDER = 5.0, 25.0, 2
BPM_CANDIDATES = [55, 65, 75, 85, 95]
ENSEMBLE_W = np.array([0.5, 0.3, 0.2], dtype=np.float32)
EINTHOVEN_BLEND_W = 0.6
MIN_VAL, MAX_VAL = 0.0, 0.09

# Image-digitization params
DEFAULT_FS = 500
DEFAULT_DURATION_S = 10.0
ASSUME_MM_PER_S = 25.0
ASSUME_MM_PER_MV = 10.0

# Reproducibility
os.environ["PYTHONHASHSEED"] = "0"
random.seed(0); np.random.seed(0)
np.set_printoptions(suppress=True, floatmode="fixed", precision=6)

print("✅ Paths:")
print(" - train.csv:", TRAIN_CSV.exists())
print(" - test.csv:", TEST_CSV.exists())
print(" - test_images dir:", TEST_IMG_DIR.exists())


## 🧮 Step 2 — Utilities (Signals)
<details><summary>Show / hide helpers</summary>

In [ ]:

def zscore(x):
    x = np.asarray(x, np.float32)
    s = np.std(x) + 1e-8
    return (x - np.mean(x)) / s

def bandpass(x, fs, lo=5.0, hi=25.0, order=2):
    if len(x) < 10: return x.astype(np.float32)
    nyq = 0.5 * fs
    lo = max(lo/nyq, 1e-3); hi = min(hi/nyq, 0.99)
    b, a = butter(order, [lo, hi], btype='band')
    return filtfilt(b, a, x).astype(np.float32)

def apply_lowpass(x, fs, cutoff=15.0, order=2):
    if len(x) <= 10: return x.astype(np.float32)
    nyq = 0.5 * fs; wn = min(cutoff/nyq, 0.99)
    b, a = butter(order, wn, btype='low')
    return filtfilt(b, a, x).astype(np.float32)

def resample_to_length(x, n):
    if len(x) == n: return x.astype(np.float32)
    return np.interp(
        np.linspace(0, 1, n, dtype=np.float32),
        np.linspace(0, 1, len(x), dtype=np.float32),
        x.astype(np.float32)
    ).astype(np.float32)

def soft_minmax_scale(x, lo=0.0, hi=0.09):
    x = np.asarray(x, np.float32)
    if x.size == 0:
        return np.full(1, (lo + hi) / 2, np.float32)
    mn, mx = float(np.nanmin(x)), float(np.nanmax(x))
    if not np.isfinite(mn) or not np.isfinite(mx) or mx <= mn:
        return np.full_like(x, (lo + hi) / 2, np.float32)
    y = (x - mn) / (mx - mn)
    return np.clip(lo + y * (hi - lo), lo, hi).astype(np.float32)

def derive_limb_leads_from_I_II(yI, yII):
    """Safely derive limb leads from I and II, resampling if needed."""
    yI = np.asarray(yI, np.float32)
    yII = np.asarray(yII, np.float32)
    # Resample to common length if mismatch
    if len(yI) != len(yII):
        n_common = min(len(yI), len(yII))
        yI = resample_to_length(yI, n_common)
        yII = resample_to_length(yII, n_common)
    III = yII - yI
    aVR = -(yI + yII) / 2.0
    aVL = yI - 0.5 * yII
    aVF = yII - 0.5 * yI
    return {'III': III, 'aVR': aVR, 'aVL': aVL, 'aVF': aVF}


def soft_blend(a, b, w):
    return (1.0 - float(w)) * a + float(w) * b


</details>

## 🩻 Step 3 — Utilities (Images)
<details><summary>Show / hide helpers</summary>

In [ ]:

def deskew_image(img_bgr):
    g = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    g = cv2.GaussianBlur(g, (3,3), 0)
    edges = cv2.Canny(g, 50, 150, apertureSize=3)
    lines = cv2.HoughLines(edges, 1, np.pi/180, 200)
    angle = 0.0
    if lines is not None and len(lines) > 0:
        thetas = [l[0][1] for l in lines if abs(l[0][1] - np.pi/2) > np.deg2rad(30)]
        if len(thetas) > 0:
            theta = float(np.median(thetas))
            angle = (theta - np.pi/2) * 180/np.pi
    (h, w) = img_bgr.shape[:2]
    M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
    return cv2.warpAffine(img_bgr, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)

def estimate_pixels_per_mm(gray):
    col = gray.mean(axis=0)
    col = (col - col.mean()) / (col.std() + 1e-6)
    ac = np.correlate(col, col, mode="full")
    ac = ac[ac.size//2:]
    peaks = np.argpartition(ac, -10)[-10:]
    peaks = np.sort(peaks)
    diffs = np.diff(peaks)
    diffs = diffs[diffs > 2]
    if len(diffs) == 0:
        return 20.0  # fallback
    return float(np.median(diffs))

def extract_waveform_mask(img_bgr):
    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    mask1 = cv2.inRange(hsv, (0,50,50), (10,255,255))
    mask2 = cv2.inRange(hsv, (170,50,50), (180,255,255))
    mask_red = cv2.bitwise_or(mask1, mask2)
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    _, mask_dark = cv2.threshold(gray, 70, 255, cv2.THRESH_BINARY_INV)
    mask = cv2.bitwise_or(mask_red, mask_dark)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, np.ones((2,2),np.uint8))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, np.ones((2,2),np.uint8))
    return mask

def split_into_leads_naive(img_bgr):
    H, W = img_bgr.shape[:2]
    panels = {}
    idx = 0
    for r in range(3):
        for c in range(4):
            x0 = int(c*W/4); x1 = int((c+1)*W/4)
            y0 = int(r*H/3); y1 = int((r+1)*H/3)
            if idx < len(LEADS):
                panels[LEADS[idx]] = img_bgr[y0:y1, x0:x1].copy()
                idx += 1
    return panels

def mask_to_timeseries(mask, fs=500, duration_s=10.0, pixels_per_mm=20.0):
    ys, xs = np.where(mask > 0)
    n_out = int(round(fs * duration_s))
    if len(xs) < 10:
        return np.zeros(n_out, np.float32)
    yx = []
    for x in range(mask.shape[1]):
        ys_col = ys[xs == x]
        yx.append(np.median(ys_col) if len(ys_col) else np.nan)
    yx = pd.Series(yx).interpolate(limit_direction="both").to_numpy(np.float32)
    s_per_mm = 1.0 / 25.0
    t_pix = np.arange(len(yx)) / float(pixels_per_mm) * s_per_mm
    v_pix = (np.nanmean(yx) - yx) / float(pixels_per_mm) * 0.1
    t_uniform = np.linspace(0.0, duration_s, n_out, dtype=np.float32)
    if np.ptp(t_pix) < 1e-6:
        return np.zeros(n_out, np.float32)
    v_uniform = np.interp(t_uniform, np.linspace(0.0, duration_s, len(v_pix), dtype=np.float32), v_pix)
    return v_uniform.astype(np.float32)


</details>

## 🫀 Step 4 — Templates from Training CSVs

In [ ]:

def build_per_lead_stats_and_beats(train_csv, train_dir, leads=LEADS):
    meta = pd.read_csv(train_csv)
    lead_vals = {ld: [] for ld in leads}
    lead_beats = {ld: [] for ld in leads}
    lead_bpm_samples = {ld: [] for ld in leads}
    for row in tqdm(meta.itertuples(index=False), total=len(meta), desc="Scan train"):
        rid = str(row.id); fs = int(row.fs)
        csvp = Path(train_dir) / rid / f"{rid}.csv"
        if not csvp.exists(): continue
        try: df = pd.read_csv(csvp)
        except: continue
        for ld in leads:
            if ld not in df.columns: continue
            y = df[ld].dropna().to_numpy(np.float32)
            if y.size < 200: continue
            lead_vals[ld].append(y)
            y_bp = bandpass(zscore(y), fs)
            iqr = np.subtract(*np.percentile(y_bp, [75,25]))
            scale = iqr if np.isfinite(iqr) and iqr>0 else np.std(y_bp)
            prominence = max(0.35*scale, 0.12)
            distance = int(max(0.32*fs,1))
            pks,_ = find_peaks(y_bp, distance=distance, prominence=prominence)
            if len(pks) < 2: continue
            rr = np.diff(pks) / float(fs)
            rr = rr[(rr>0.3)&(rr<2.0)]
            if rr.size >= 1:
                lead_bpm_samples[ld].append(float(np.clip(60.0/np.median(rr), 40.0, 160.0)))
            n_pre, n_post = int(round(0.20*fs)), int(round(0.40*fs))
            for pk in pks:
                a, b = pk-n_pre, pk+n_post
                if a < 0 or b >= len(y): continue
                seg = y[a:b+1].astype(np.float32)
                lead_beats[ld].append(resample_to_length(seg, 360))

    lead_stats = {}
    for ld in leads:
        if len(lead_vals[ld]) == 0:
            lead_stats[ld] = {'mean':0.0,'std':0.1}
        else:
            vals = np.concatenate(lead_vals[ld]).astype(np.float32)
            lead_stats[ld] = {'mean': float(np.mean(vals)), 'std': float(np.std(vals))}
    return lead_stats, lead_beats, lead_bpm_samples

def build_lead_templates(lead_beats, leads=LEADS):
    lead_template = {}
    for ld in leads:
        if len(lead_beats[ld]) > 0:
            arr = np.vstack(lead_beats[ld]).astype(np.float32)
            tpl = np.median(arr, axis=0).astype(np.float32)
        else:
            t = np.linspace(0, 1, 360, dtype=np.float32)
            tpl = np.sin(2*np.pi*t).astype(np.float32)
        lead_template[ld] = zscore(tpl)
    return lead_template


In [ ]:

print("⚙️ [1/6] Scanning training to build templates...")
lead_stats, lead_beats, lead_bpms = build_per_lead_stats_and_beats(TRAIN_CSV, TRAIN_DIR, LEADS)
lead_template = build_lead_templates(lead_beats, LEADS)
mean_templates = {ld: (np.mean(np.vstack(v), axis=0) if len(v)>0 else lead_template.get('II')) 
                  for ld, v in lead_beats.items()}
per_lead_bpm_prior = {ld: (float(np.median(v)) if len(v)>0 else 75.0) for ld, v in lead_bpms.items()}
print("Templates built for", len(lead_template), "leads.")


## 💫 Step 5 — Reconstruction (BPM tiling + Ensemble + Einthoven)
<details><summary>Show / hide code</summary>

In [ ]:

def tile_template(template_beat, fs, n_out, bpm, amp=1.0):
    beat_samples = max(4, int(round((60.0 / max(bpm, 1e-6)) * fs)))
    one = np.interp(
        np.linspace(0, 1, beat_samples, dtype=np.float32),
        np.linspace(0, 1, len(template_beat), dtype=np.float32),
        template_beat
    ).astype(np.float32)
    reps = int(np.ceil(n_out / len(one)))
    y = np.tile(one, reps)[:n_out]
    y = zscore(y) * float(amp)
    return y.astype(np.float32)

def _nextpow2(n):
    return 1 << (int(math.ceil(math.log2(max(1, n)))))

def autocorr_peak_score(y, fs, min_rr_s=0.35, max_rr_s=1.5):
    y = zscore(y).astype(np.float32); n = len(y)
    if n < 16: return 0.0
    m = _nextpow2(2*n - 1)
    Y = np.fft.rfft(y, n=m)
    ac_full = np.fft.irfft(Y * np.conj(Y), n=m)
    ac = ac_full[:n]
    lo = int(max(min_rr_s * fs, 1)); hi = int(min(max_rr_s * fs, n - 1))
    if hi <= lo: return 0.0
    seg = ac[lo:hi]
    if seg.size == 0: return 0.0
    peak = float(seg.max()); norm = float(ac[0]) + 1e-8
    return float(np.clip(peak / norm, 0.0, 1.0))

def choose_best_bpm(template_beat, fs, n_out, bpm_list=[55,65,75,85,95]):
    best_bpm, best_score, best_y = None, -1.0, None
    for bpm in bpm_list:
        y = tile_template(template_beat, fs, n_out, bpm, amp=1.0)
        sc = autocorr_peak_score(y, fs)
        if sc > best_score:
            best_bpm, best_score, best_y = bpm, sc, y
    return best_bpm, best_y


</details>

## 🩻 Step 6 — Image Digitization (OpenCV + optional model hook)
<details><summary>Show / hide code</summary>

In [ ]:

USE_SEGMENTATION_MODEL = False  # Set True when you attach your weights and implement the hook.

def init_segmentor_if_available():
    if not USE_SEGMENTATION_MODEL:
        return None
    # Example:
    # from my_digitizer import load_model
    # return load_model("/kaggle/input/my-weights/model.pth")
    return None

def segment_mask_with_model_or_classical(segmentor, img_bgr):
    if segmentor is None:
        return extract_waveform_mask(img_bgr)
    # Example:
    # return segmentor.predict_mask(img_bgr)
    return extract_waveform_mask(img_bgr)

def digitize_image_to_leads(img_bgr, fs=500, duration_s=10.0, einthoven_blend_w=0.0, segmentor=None):
    img = deskew_image(img_bgr)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ppm = estimate_pixels_per_mm(gray)
    panels = split_into_leads_naive(img)
    traces = {}
    for ld, pimg in panels.items():
        mask = segment_mask_with_model_or_classical(segmentor, pimg)
        y = mask_to_timeseries(mask, fs=fs, duration_s=duration_s, pixels_per_mm=ppm)
        y = apply_lowpass(zscore(y), fs)
        traces[ld] = y
    if einthoven_blend_w > 0.0 and ("I" in traces) and ("II" in traces):
        derived = derive_limb_leads_from_I_II(traces["I"], traces["II"])
        for dlead, ydrv in derived.items():
            ydrv = zscore(ydrv)
            if dlead in traces and len(traces[dlead]) == len(ydrv):
                traces[dlead] = soft_blend(traces[dlead], ydrv, einthoven_blend_w)
            else:
                traces[dlead] = ydrv
    for ld in list(traces.keys()):
        traces[ld] = soft_minmax_scale(traces[ld], 0.0, 0.09)
    return traces


</details>

## 🔀 Step 7 — Hybrid Blending

In [ ]:

def hybrid_blend(digitized, reconstructed, w_digitized=0.6):
    out = {}
    for ld in LEADS:
        yd = digitized.get(ld, None)
        yr = reconstructed.get(ld, None)
        if yd is None and yr is None:
            continue
        if yd is None:
            out[ld] = yr
        elif yr is None:
            out[ld] = yd
        else:
            n = max(len(yd), len(yr))
            yd_rs = resample_to_length(yd, n)
            yr_rs = resample_to_length(yr, n)
            out[ld] = soft_minmax_scale(soft_blend(yr_rs, yd_rs, w_digitized), 0.0, 0.09)
    return out


## 📦 Step 8 — Predict & Create `submission.csv

In [ ]:

print("⚙️ [2/6] Loading test metadata...")
test_df = pd.read_csv(TEST_CSV)
by_record = {}
for r in test_df.itertuples(index=False):
    by_record.setdefault(int(r.id), []).append(r)

segmentor = init_segmentor_if_available()
pred = {}

print("⚙️ [3/6] Digitizing images & reconstructing...")
for rid, items in tqdm(by_record.items(), desc="Records"):
    # Locate test image
    img_path = None
    jpg = TEST_IMG_DIR / f"{rid}.jpg"
    png = TEST_IMG_DIR / f"{rid}.png"
    if jpg.exists(): img_path = jpg
    elif png.exists(): img_path = png

    # Digitization path
    if img_path is not None:
        img = cv2.imread(str(img_path))
        digitized_traces = digitize_image_to_leads(img, fs=500, duration_s=10.0, einthoven_blend_w=0.6, segmentor=segmentor)
    else:
        digitized_traces = {}

    # Reconstruction path
    recon_traces = {}
    for r in items:
        lead = str(r.lead); fs = int(r.fs); n = int(r.number_of_rows)
        tpl = lead_template.get(lead, lead_template.get("II"))
        best_bpm, y_best = choose_best_bpm(tpl, fs, n)
        y_fixed = tile_template(tpl, fs, n, per_lead_bpm_prior.get(lead, 75.0))
        y_mean  = resample_to_length(mean_templates.get(lead, mean_templates.get("II")), n)
        B,F,M = zscore(apply_lowpass(y_best,fs)), zscore(apply_lowpass(y_fixed,fs)), zscore(apply_lowpass(y_mean,fs))
        w = np.array([0.5,0.3,0.2], dtype=np.float32); w = w/(np.sum(w)+1e-8)
        y_syn = (w[0]*B + w[1]*F + w[2]*M).astype(np.float32)
        recon_traces[lead] = soft_minmax_scale(y_syn, 0.0, 0.09)

    if ('I' in recon_traces) and ('II' in recon_traces) and 0.6>0:
        derived = derive_limb_leads_from_I_II(recon_traces['I'], recon_traces['II'])
        for dlead, ydrv in derived.items():
            if dlead in recon_traces:
                recon_traces[dlead] = soft_blend(recon_traces[dlead], zscore(ydrv), 0.6)
            else:
                recon_traces[dlead] = zscore(ydrv)

    # Hybrid
    traces = hybrid_blend(digitized_traces, recon_traces, w_digitized=0.6)

    # Store resampled to request
    for r in items:
        lead = str(r.lead); n = int(r.number_of_rows)
        y = traces.get(lead, np.zeros(n, np.float32))
        if len(y) != n:
            y = resample_to_length(y, n)
        pred[(rid, lead)] = soft_minmax_scale(y, 0.0, 0.09)

print("⚙️ [4/6] Writing submission.csv ...")
rows = []
for r in test_df.itertuples(index=False):
    rid, lead, n = int(r.id), str(r.lead), int(r.number_of_rows)
    y = pred.get((rid, lead), np.zeros(n, np.float32))
    if len(y) != n: y = resample_to_length(y, n)
    for i in range(n):
        rows.append((f"{rid}_{i}_{lead}", float(y[i])))

sub = pd.DataFrame(rows, columns=["id","value"])
sub_path = WORK_DIR / "submission.csv"
sub.to_csv(sub_path, index=False)
print("✅ Done:", sub_path)
sub.head()


In [ ]:
# ✅ Final check for Kaggle submission file
import os
sub_path = "/kaggle/working/submission.csv"

if os.path.exists(sub_path):
    import pandas as pd
    df = pd.read_csv(sub_path)
    print(f"✅ submission.csv found ({len(df)} rows)")
    display(df.head(10))
else:
    print("❌ submission.csv not found — please check earlier errors.")


## 👀 Step 9 — Visual Checks {#viz}

In [ ]:

import matplotlib.pyplot as plt

some_id = None
for rid in by_record.keys():
    jpg = TEST_IMG_DIR / f"{rid}.jpg"
    png = TEST_IMG_DIR / f"{rid}.png"
    if jpg.exists() or png.exists():
        some_id = rid; break

if some_id is not None:
    leads_to_plot = ["I","II","V1","V5"]
    plt.figure(figsize=(12,4))
    for ld in leads_to_plot:
        key = (some_id, ld)
        if key in pred:
            plt.plot(pred[key], label=ld)
    plt.title(f"Hybrid predictions (record {some_id})")
    plt.legend()
    plt.show()
else:
    print("No example test image found for quick plotting preview.")



## 📝 Step 10 — Notes & Next Steps {#next}

**Why this hybrid works:**  
- ECGs are quasi-periodic; a **median beat** is a strong prior.  
- The **image digitizer** recovers subject-specific morphology from the paper scan.  
- The **hybrid** balances physics-based priors with actual observed morphology.

**Limitations:**  
- Naive 3×4 panel split; replace with YOLO/FRCNN detector for robust layouts.  
- Grid spacing estimated heuristically; add calibration pulse detection for precise mV/time scaling.  
- Optional U-Net/nnU-Net segmentor not included here (hook provided).

**Upgrades:**  
- Train a segmentation model on ECG-Image-Kit / ECG-Image-Database and enable `USE_SEGMENTATION_MODEL`.  
- Tempo-tracking BPM (multi-segment autocorrelation) for rate changes.  
- Rhythm-strip parsing and vendor-specific panel templates.

---

❤️ If you found this helpful, an **upvote** would be amazing.  
Happy Kaggle-ing! 🚀
